# Wikipics demo

In [1]:
from pathlib import Path
from typing import NamedTuple
from IPython.display import display
from ipywidgets.widgets import Image, Layout

import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

In [2]:
def fetch(url) -> ImageRecord:
    resp = httpx.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

In [3]:
url = get_sample_url(1_000_000)
img_rec = fetch(url)
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

   1_024_408 bytes | Joseph_Wright_of_Derby_-_Dovedale_by_Moonlight_-_Google_Art_Project.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x02\x00&\x00&\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\…

In [ ]:
%%time
img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0
for url in urls:
    img_rec = fetch(url)
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x02\x00&\x00&\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\…

   2_019_580 bytes | Edvard_Munch_-_The_Kiss_-_Google_Art_Project.jpg


Image(value=b'\xff\xd8\xff\xe1%\xfeExif\x00\x00II*\x00\x08\x00\x00\x00\t\x00\x0f\x01\x02\x00\x06\x00\x00\x00z\…

   1_975_417 bytes | Dolceacqua43_-_Artista_locale_mentre_dipinge_un_acquarello.jpg


Image(value=b'\xff\xd8\xff\xe1d*Exif\x00\x00MM\x00*\x00\x002\xf2\x00\x1c\x82\x9a\x00\x05\x00\x00\x00\x01\x00\x…

...

   1_969_876 bytes | Yogapith%2C_Mayapur.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\xf0\x00\xf0\x00\x00\xff\xe1\x1e=Exif\x00\x00II*…

   1_974_653 bytes | Phascolarctos_cinereus_Bonorong.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\x96\x00\x96\x00\x00\xff\xe1DvExif\x00\x00MM\x00…

   1_976_293 bytes | United_States_Declaration_of_Independence.jpg


In [ ]:
from gallery import Gallery

In [ ]:
%%time
num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)

for i, url in enumerate(urls):
    img_rec = fetch(url)
    total_bytes += img_rec.size
    gallery.update(i, img_rec.pixels, url)

print(f'TOTAL BYTES: {gallery.size:_}')

In [ ]:
%%time
from concurrent import futures

num_images = 20
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)
with futures.ThreadPoolExecutor() as pool:
    img_records = pool.map(fetch, urls)
    for i, img_rec in enumerate(img_records):
        gallery.update(i, img_rec.pixels, img_rec.name)

print(f'TOTAL BYTES: {gallery.size:_}')

In [ ]:
%%time
from concurrent import futures

num_images = 20
gallery = Gallery(num_images)
gallery.display()
urls = get_sample_urls(2_000_000, num_images)

with futures.ThreadPoolExecutor() as pool:
    tasks = [pool.submit(fetch, url) for url in urls]
    for i, future in enumerate(futures.as_completed(tasks)):
        img_rec = future.result()
        gallery.update(i, img_rec.pixels, img_rec.name)
        
print(f'TOTAL BYTES: {gallery.size:_}')